<a href="https://colab.research.google.com/github/RichardFreedman/CRIM-Project-RF/blob/master/CRIM_Intervals_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Load CRIM Intervals Software

In [12]:
import shutil 
from pathlib import Path
intervals = Path('intervals')
if intervals.exists():
    shutil.rmtree(intervals)
    !git clone https://github.com/HCDigitalScholarship/intervals.git
    !pip install httpx
else:
    !git clone https://github.com/HCDigitalScholarship/intervals.git
    !pip install httpx

Cloning into 'intervals'...
remote: Enumerating objects: 171, done.
remote: Counting objects: 100% (171/171), done.
remote: Compressing objects: 100% (129/129), done.
remote: Total 171 (delta 83), reused 113 (delta 39), pack-reused 0
Receiving objects: 100% (171/171), 67.65 KiB | 5.20 MiB/s, done.
Resolving deltas: 100% (83/83), done.


In [13]:
from intervals.main_objs import *

## Start CRIM Intervals

## Load MEI Files from CRIM or Github by pasting one or more of [these links](https://docs.google.com/spreadsheets/d/1TzRqnzgcYYuQqZR78c5nizIsBWp4pnblm2wbU03uuSQ/edit?auth_email=rfreedma@haverford.edu#gid=0) below.

*Note:  each file must be in quotation marks and separated by commas







In [14]:
corpus = CorpusBase(['https://crimproject.org/mei/CRIM_Model_0008.mei', 'https://crimproject.org/mei/CRIM_Mass_0005_5.mei'])


Requesting file from https://crimproject.org/mei/CRIM_Model_0008.mei...
Successfully imported.
Requesting file from https://crimproject.org/mei/CRIM_Mass_0005_5.mei...
Successfully imported.


## Give the scores short names, in order according to the way they were listed above

In [15]:
model, mass = corpus.scores

## Now apply various methods to the scores:
* **getNoteRest** returns all the notes and rests, each voice as a column
* **getDuration** returns the durations for all notes and rests, as above
* **getMelodic** returns the melodic intervals in each voice as a column
* **getHarmonic** returns pairs of harmonic intervals between each pair of voices
* **getNgrams**  returns segments of various kinds, melodic (one voice) or modular (pairs of voices, including vertical and horizontal motion)




---

### Pandas Tools:
* **df.value_counts()**  returns summary for each pitch, duration for any type
* save results as variable, then:**`.apply(pd.Series.value_counts).fillna(0).astype(int) `**


---

### Documentation available via this command:
* for any method, use the following read documentation:
`print(model.getNgrams.__doc__)`

---








In [ ]:
print(model.getMelodic.__doc__)

Return melodic intervals for all voice pairs. Each melodic interval
        is associated with the starting offset of the second note in the
        interval. 

        :param str kind: use "d" for diatonic intervals without quality, "q"
            (default) for diatonic intervals with quality, or "s" for semitonal
            intervals. Only the first character is used, and it's case
            insensitive.
        :param bool directed: defaults to True which shows that the voice that
            is lower on the staff is a higher pitch than the voice that is
            higher on the staff. This is desginated with a "-" prefix.
        :param bool compound: whether to use compound (True, default) or simple
            (False) intervals. In the case of simple diatonic intervals, it
            simplifies to within the octave, so octaves don't get simplified to
            unisons. But for semitonal intervals, an interval of an octave
            (12 semitones) would does get simplifi

# Notes and Rests

In [23]:
notes = model.getNoteRest()
notes.fillna(value= "-", inplace=True)
notes.reset_index()


,index,[Superius],Altus,Tenor,Bassus
0,0.0,G4,Rest,Rest,Rest
1,4.0,C5,-,-,-
2,8.0,-,Rest,Rest,Rest
3,12.0,C5,-,-,-
4,16.0,D5,G3,Rest,Rest
...,...,...,...,...,...
553,1256.0,-,-,D4,G3
554,1268.0,C5,G4,-,-
555,1272.0,-,-,C4,C3
556,1284.0,C5,G4,-,-


In [17]:
notes.value_counts()
# notes.stack().value_counts()


[Superius]  Altus  Tenor  Bassus
-           C4     -      -         23
C5          -      -      -         17
-           E4     -      -         16
B4          -      -      -         15
-           D4     -      -         14
                                    ..
D5          -      F4     D3         1
                   D4     -          1
                   B3     -          1
                   A3     F3         1
C5          A3     Rest   C3         1
Length: 281, dtype: int64

In [27]:
df = notes.apply(pd.Series.value_counts).fillna(0).astype(int)


# Melodic Intervals
* kind='d' for diatonic; 's' for chromatic/semitone
* To save as CSV:  
`mel_int.to_csv('file_name.csv')`

In [24]:
mel_int = mass.getMelodic(kind='d')
mel_int.fillna(value= "-", inplace=True)
mel_int



,Sup[erius],Altus,Tenor,Bassus
0.0,-,Rest,Rest,Rest
4.0,4,-,-,-
8.0,-,Rest,Rest,Rest
12.0,1,-,-,-
16.0,2,-,Rest,Rest
...,...,...,...,...
706.0,1,-2,-,-
707.0,-,-2,-,-
708.0,-,2,-,-
710.0,-2,-,-,-


# Durations

In [26]:
durs = mass.getDuration()
durs.fillna(value= "-", inplace=True)



-       828
1.0     211
4.0     174
8.0     149
2.0     148
3.0      54
6.0      21
16.0      9
24.0      3
0.5       2
12.0      1
dtype: int64

## Combine Notes and Durations as One DataFrame

In [9]:
notes_durs = pd.concat([notes, durs], axis=1)
notes_durs


,[Superius],Altus,Tenor,Bassus,Sup[erius],Altus,Tenor,Bassus
0.0,G4,Rest,Rest,Rest,4,8,8,8
4.0,C5,-,-,-,8,-,-,-
8.0,-,Rest,Rest,Rest,-,8,8,8
12.0,C5,-,-,-,4,-,-,-
16.0,D5,G3,Rest,Rest,4,4,8,8
...,...,...,...,...,...,...,...,...
1256.0,-,-,D4,G3,NaN,NaN,NaN,NaN
1268.0,C5,G4,-,-,NaN,NaN,NaN,NaN
1272.0,-,-,C4,C3,NaN,NaN,NaN,NaN
1284.0,C5,G4,-,-,NaN,NaN,NaN,NaN


# Select Columns for One Voice

In [10]:
notes_durs_s = notes_durs.iloc[:, [0,4]]
notes_durs_s

,[Superius],Sup[erius]
0.0,G4,4
4.0,C5,8
8.0,-,-
12.0,C5,4
16.0,D5,4
...,...,...
1256.0,-,NaN
1268.0,C5,NaN
1272.0,-,NaN
1284.0,C5,NaN


## N-Grams in Each Voice
* for Melodic or Durations

In [39]:
mel = model.getMelodic()
ngrams = model.getNgrams(df=mel, n=5)
ngrams
# mel2 = mel.iloc[:, [0]]
# ngrams = model.getNgrams(df=mel2, n=5)
# out = ngrams.value_counts()
# out





,[Superius],Altus,Tenor,Bassus
4.0,"P4, P1, M2, M2, -M3",NaN,NaN,NaN
20.0,NaN,"P4, P1, M2, M2, -M3",NaN,NaN
36.0,NaN,NaN,"P4, P1, M2, M2, -M3",NaN
52.0,NaN,NaN,NaN,"P4, P1, M2, M2, -M3"
62.0,"-m2, -M2, -M2, M2, -M2",NaN,NaN,NaN
...,...,...,...,...
1208.0,NaN,NaN,"-m2, P1, -M2, M2, m2","P5, P1, -M2, -m2, -M2"
1212.0,NaN,NaN,NaN,"P1, -M2, -m2, -M2, -M2"
1236.0,"P1, -M2, -M2, -m2, m2","P1, P1, -m3, m2, M2",NaN,NaN
1240.0,"-M2, -M2, -m2, m2, P1","P1, -m3, m2, M2, P1","P1, -m2, m2, M2, -M2","P8, -P4, M2, -M2, -P5"


# Harmonic Intervals between Voices

In [47]:
harm = model.getHarmonic()


,Bassus_Tenor,Bassus_Altus,Bassus_[Superius],Tenor_Altus,Tenor_[Superius],Altus_[Superius]
-,169,259,295,227,276,189
-A4,0,0,0,2,0,0
-M2,0,2,0,8,0,0
-M3,0,0,0,25,0,0
-M6,0,0,0,7,0,0
-P4,0,0,0,30,0,1
-P5,0,0,0,21,0,1
-P8,0,0,0,13,0,0
-d5,0,0,0,1,0,0
-m2,0,2,0,2,0,0


# Two-Voice Modules as N-Grams

In [41]:
modules = model.getNgrams(how='modules', cell_type=str)
mod = modules.iloc[:, 0:6]
mod
# mod.apply(pd.Series.value_counts).fillna(0).astype(int)


,Bassus_Tenor,Bassus_Altus,Bassus_[Superius],Tenor_Altus,Tenor_[Superius],Altus_[Superius]
16.0,NaN,NaN,NaN,NaN,NaN,"12_2, 10_-3, 8"
20.0,NaN,NaN,NaN,NaN,NaN,"10_-3, 8_Held, 8"
32.0,NaN,NaN,NaN,"5_Held, 3_Rest, 1",NaN,NaN
36.0,NaN,NaN,NaN,"3_Rest, 1_Held, 1",NaN,NaN
48.0,"12_Held, 10_Held, 8",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
1248.0,"3_-2, 3_Held, 5","6_-2, 7_Held, 8","10_-2, 9_Held, 10","4_-2, 5_Held, 4","8_-2, 7_Held, 6","5_-2, 3_Held, 3"
1252.0,"3_Held, 5_2, 5","7_Held, 8_2, 8","9_Held, 10_2, 11","5_Held, 4_2, 4","7_Held, 6_2, 7","3_Held, 3_2, 4"
1256.0,"5_2, 5_Held, 8","8_2, 8_Held, 12","10_2, 11_Held, 15","4_2, 4_Held, 5","6_2, 7_Held, 8","3_2, 4_Held, 4"
1268.0,"5_Held, 8_1, 8","8_Held, 12_1, 12","11_Held, 15_1, 15","4_Held, 5_1, 5","7_Held, 8_1, 8","4_Held, 4_1, 4"
